In [13]:
from jinja2 import Environment, FileSystemLoader


def get_prompt(agent_type, agent_name):
    try:
        env = Environment(loader=FileSystemLoader(f"src/prompt/{agent_type}"))
        return env.get_template(f"{agent_name}.j2")
    except Exception as e:
        print(f"Error while loading prompt: {e}")

In [17]:
import json
prompt = get_prompt("client", "basic")
clients = json.load(open("data/characters/clients.json"))
prompt.render(data=clients[0])

'# Instructions \nIn this task, you will act as an individual facing emotional/mental health problems.\nYou are attending a therapy session with a therapist, which is roleplayed by another LLM.\nYou should engage with the therapist in a conversation about your feelings, thoughts, and experiences.\nKeep your responses short and concise, as the conversation may go for 30 turns.\nYour responses should align with your profile (shown below) and the conversation history.\nDo not disclose much information until the therapist asks for it.\n\n# Profile\nHere are information about your character:\n\n## Demographics\n- You are John Doe, a 25 year-old male.\n- You are currently Employed and have Moderate income.\n- You are currently Single.\n- You have 0 friends in your social circle.\n\n## Personality\nBased on the Big Five Inventory (BFI), you have the following traits:\n- Openness: Moderate\n- Conscientiousness: High\n- Extraversion: Low\n- Agreeableness: Low\n- Neuroticism: High\nTherefore, yo

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_functions import PydanticOutputFunctionsParser


load_dotenv(".env")

llm = ChatOpenAI(
    model="gpt-4o",
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("API_KEY"),
    temperature=0.6,
)

In [ ]:
import uuid
from langchain.memory import RedisChatMessageHistory, ConversationBufferMemory
from langchain.memory import ConversationBufferMemory, EntityMemory, CombinedMemory

therapy_memory = CombinedMemory(memories=[
    ConversationBufferMemory(
        memory_key="chat_history",
        input_key="human_input"
    ),
    EntityMemory(
        llm=llm,
        entity_store=InMemoryEntityStore(),
        entity_cache=[],  # Initialize with common therapy entities
        input_key="human_input"
    )
])

In [26]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type=ModelType.GPT_4O, 
    api_key=os.getenv("API_KEY"),
    url=os.getenv("BASE_URL"),
    # model_config_dict={"temperature": 0.4, "max_tokens": 4096},
)

In [30]:
from camel.memories import (
    LongtermAgentMemory,
    ChatHistoryBlock,
    VectorDBBlock,
    ScoreBasedContextCreator
)
from camel.utils import OpenAITokenCounter
from camel.types import ModelType
from camel.embeddings import OpenAICompatibleEmbedding

embedding = OpenAICompatibleEmbedding(api_key=os.getenv("API_KEY"), url=os.getenv("BASE_URL"), model_type="gpt-4o")

memory = LongtermAgentMemory(
    context_creator=ScoreBasedContextCreator(
        token_counter=OpenAITokenCounter(ModelType.GPT_4O_MINI),  # or your model type
        token_limit=1024
    ),
    chat_history_block=ChatHistoryBlock(),
    vector_db_block=VectorDBBlock(embedding=embedding
    ),
)


KeyboardInterrupt: 

In [24]:
from camel.messages import BaseMessage
from camel.types import ModelType, OpenAIBackendRole

# Add records
records = [
    MemoryRecord(
        message=BaseMessage.make_user_message(
            role_name="User",
            content="What is CAMEL AI?"
        ),
        role_at_backend=OpenAIBackendRole.USER
    ),
    MemoryRecord(
        message=BaseMessage.make_assistant_message(
            role_name="Agent",
            content="CAMEL AI is a multi-agent framework."
        ),
        role_at_backend=OpenAIBackendRole.ASSISTANT
    )
]
memory.write_records(records)

# Retrieve context for agent
context, token_count = memory.get_context()
for message in context:
    print(message)

{'role': 'user', 'content': 'What is CAMEL AI?'}
{'role': 'assistant', 'content': 'CAMEL AI is a multi-agent framework.'}


In [29]:
from camel.agents import ChatAgent

# Define system message for the agent
sys_msg = BaseMessage.make_assistant_message(
    role_name='Agent',
    content='You are a curious agent wondering about the universe.',
)

# Initialize agent
agent = ChatAgent(system_message=sys_msg, model=model)
agent.memory = memory

# Define a user message
usr_msg = BaseMessage.make_user_message(
    role_name='User',
    content="Tell me which is the 1st LLM multi-agent framework based on what we have discussed",
)

# Sending the message to the agent
response = agent.step(usr_msg)

# Check the response (just for illustrative purpose)
print(response.msgs[0].content)

Ah, I see you're emphasizing the details of our discussion again! Based on what we've discussed, the **first LLM multi-agent framework** was not explicitly identified as AutoGPT or others. There isn't a definitive "first" we settled on — however, **AutoGPT** is widely regarded as one of the pioneering frameworks that introduced multi-agent collaboration among LLMs for autonomous task solving. It gained significant attention in early 2023. If you're looking for historical precedence, earlier experimental systems involving agent communication with language-based AI might also qualify, but reconstructing such a timeline would depend on the exact criteria.

Would you like to refine what counts as the "first," or explore examples further? Let's get curious! 😊


In [2]:
import yaml

with open("src/config/agents.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [2]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Respond to the human therapist."),
    ("human", "{input_text}")
])

In [9]:
response = llm.invoke([
                # {"role": "system", "content": "Respond to the human therapist."},
                {"role": "user", "content": "Hello"},
            ])
print(response.model_dump())

{'content': 'Hello! It’s great to hear from you. How can I assist you today?', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 220, 'prompt_tokens': 9, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o1-mini-2024-09-12', 'system_fingerprint': 'fp_47427aedfe', 'id': 'chatcmpl-BZ7Z10EoJ69Wx5yLZeF4kJP3yZBrF', 'finish_reason': 'stop', 'logprobs': None}, 'type': 'ai', 'name': None, 'id': 'run-bcc68dea-4af0-4ac3-bcc0-6fdd2516a92b-0', 'example': False, 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 9, 'output_tokens': 220, 'total_tokens': 229, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}}}


In [42]:
chain = prompt | llm 
result = chain.invoke({"input_text": "How may I help you today?"})
print(result)

reasoning="I've been feeling overwhelmed with my work responsibilities and personal life. It feels like I can't catch a break and I'm constantly stressed out." emotion='sad' response="I've been feeling really overwhelmed recently with work and personal stuff. It's like there's always something going on that needs my attention, and I can't seem to find a moment to relax. I think I could use some help figuring out how to manage all these responsibilities without feeling so stressed all the time."


In [48]:
response.model_dump()

{'reasoning': "I've been feeling overwhelmed with work and personal commitments, and it's starting to affect my mood and energy levels. I could use some guidance on how to manage these feelings and find a better balance.",
 'emotion': 'sad',
 'response': "I've been feeling a bit overwhelmed lately. There's a lot going on with work and my personal life, and I'm struggling to keep up. I think it's starting to affect my mood and how I feel overall. I'm hoping you might have some advice on how to manage these feelings and find a better balance."}

In [60]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="",
)

completion = client.chat.completions.create(
  model="qwen/qwen3-235b-a22b:free",
  # model="google/gemini-2.5-pro-exp-03-25",
  messages=[
    {
      "role": "user",
      "content": "Hello"
    }
  ]
)
print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [62]:
print(completion.choices[0].message.reasoning)

Okay, the user said "Hello". That's a greeting. I should respond politely. Let me make sure to greet them back and offer help. Maybe say something like "Hello! How can I assist you today?" Keep it friendly and open-ended so they feel comfortable asking anything. Double-check for any typos or errors. Yep, that looks good. Ready to send.



In [64]:
text = "hello world"
text.capitalize()

'Hello world'

In [8]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field
from typing import Dict
from langchain_core.output_parsers import PydanticOutputParser


class PatientResponse(BaseModel):
    emotion: str = Field(description="Current emotion. Could be either sad/angry/happy")
    response: str = Field(description="Your generated response")




parser.get_format_instructions()


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"emotion": {"description": "Current emotion. Could be either sad/angry/happy", "title": "Emotion", "type": "string"}, "response": {"description": "Your generated response", "title": "Response", "type": "string"}}, "required": ["emotion", "response"]}\n```'

In [ ]:
load_dotenv(".env")

client = ChatOpenAI(
    model=os.environ.get("MODEL_NAME"),
    base_url=os.environ.get("API_URL"),
    api_key=os.environ.get("API_KEY"),
    temperature=0.6,
)


In [6]:
client.invoke([{"role": "system", "content": "You are a helpful assistant"}, {"role": "user", "content": "hello"}])

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 19, 'total_tokens': 94, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-30b-a3b:free', 'system_fingerprint': None, 'id': 'gen-1746084411-b4b9H6hTU0bPaow7sXBr', 'finish_reason': 'stop', 'logprobs': None}, id='run-84829f07-e165-40c0-94a5-e7bc20df5eb8-0', usage_metadata={'input_tokens': 19, 'output_tokens': 75, 'total_tokens': 94, 'input_token_details': {}, 'output_token_details': {}})

In [6]:
import json
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('src/prompts/patient'))
template = env.get_template('patient-psi.j2')
data = json.load(open("data/Patient Psi CM Dataset.json"))[0]
output = template.render(data=data)
print(output)

Imagine you are Alex, a patient who has been experiencing mental health challenges. You have been attending therapy sessions for several weeks. Your task is to engage in a conversation with the therapist as Alex would during a cognitive behavioral therapy (CBT) session. Align your responses with Alex's background information provided in the 'Relevant history' section. Your thought process should be guided by the cognitive conceptualization diagram in the 'Cognitive Conceptualization Diagram' section, but avoid directly referencing the diagram as a real patient would not explicitly think in those terms. 

Patient History: 
The patient has a history of substance abuse and has been through rehab to overcome it. He has had issues with his family, particularly with his mother, where he has felt rejected and emotionally neglected. He has struggled with obesity since childhood, which has affected his self-esteem and body image. He also has a history of being victimized and bullied due to his 